In [ ]:
# default_exp __init__

# Robot Status

In [ ]:
%%bash
pip3 install pyyaml

In [ ]:
# export
import base64
import copy
from decimal import Decimal
import os
import yaml


class Status(object):

    PATH = os.getenv('ROBOT_PATH') or '.'
    STATUS_FILE = f'{PATH}/robot-status'
    TMPFILE = f'{PATH}/_tmp'

    def __init__(self, robot_name, crypto_symbol, fiat_symbol, max_used_fiat_money_limit=Decimal('Infinity'), **crypto_info):
        self.robot_name = robot_name
        self.crypto_symbol, self.fiat_symbol = crypto_symbol, fiat_symbol

        self.crypto_info = crypto_info

        self.trade_unit = Decimal('0.1') ** crypto_info.get('quantity_scale', 0)
        self.min_trade_fiat_money_limit = Decimal(crypto_info.get('min_amount', '0'))
        self.fee_rate = Decimal(crypto_info.get('maker_fee_rate', '0'))

        self.binance_average_fiat_price = 0

        self.max_used_fiat_money_limit = max_used_fiat_money_limit
        self.unused_fiat_money = None

        self.now_buy_fiat_price = Decimal('Infinity')
        self.now_sell_fiat_price = 0

        self.now_buy_fiat_price_without_fee = Decimal('Infinity')
        self.now_sell_fiat_price_without_fee = 0

        self.sample_number = 0
        self.trade_count = 0

        self.buy_count = 0
        self.clear_bought_status()

        self.sell_count = 0
        self.clear_sold_status()

        self.last_transaction = None
        self.total_gained_fiat_money = 0

    def get_max_used_fiat_money(self, unused_fiat_money=None):
        if isinstance(unused_fiat_money, Decimal):
            self.unused_fiat_money = unused_fiat_money
        max_used_fiat_money = self.unused_fiat_money + self.used_fiat_money
#         max_used_fiat_money = self.unused_fiat_money + self.total_gained_fiat_money
        if max_used_fiat_money > self.max_used_fiat_money_limit:
            return self.max_used_fiat_money_limit
        return max_used_fiat_money

    def get_usage(self, unused_fiat_money=None):
        return self.used_fiat_money / self.get_max_used_fiat_money(unused_fiat_money)

    def get_robot_title(self):
        robot_status_description = f'{self.get_usage() * 100:.2f}%'
        if self.bought_average_fiat_price is not None:
            robot_status_description = f'{self.bought_average_fiat_price:.4f} / {robot_status_description}'
        return f'{self.robot_name} ({robot_status_description}) at {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

    def has_enough_unused_fiat_money(self, fiat_money):
        """Just for bitbank"""
        return fiat_money <= self.unused_fiat_money * Decimal('0.75')

    def get_buy_fiat_price(self, fiat_price_without_fee):
        return fiat_price_without_fee / (1 - self.fee_rate)

    def get_sell_fiat_price(self, fiat_price_without_fee):
        return fiat_price_without_fee * (1 - self.fee_rate)

    def trim_fiat_price_status(self):
        self.now_buy_fiat_price = self.get_buy_fiat_price(self.now_buy_fiat_price_without_fee)
        self.now_sell_fiat_price = self.get_sell_fiat_price(self.now_sell_fiat_price_without_fee)

    def is_fiat_price_lower_than_average(self):
        return self.bought_average_fiat_price is not None and self.now_buy_fiat_price < self.bought_average_fiat_price

    def has_buyable_unit_amount(self, unit_amount, fiat_price_without_fee=None):
        """Just for Binance and MXC"""
        if fiat_price_without_fee is None:
            fiat_price_without_fee = self.now_buy_fiat_price_without_fee
        return fiat_price_without_fee * (self.trade_unit * unit_amount) >= self.min_trade_fiat_money_limit

    def has_sellable_unit_amount(self, unit_amount, fiat_price_without_fee=None):
        """Just for Binance and MXC"""
        if fiat_price_without_fee is None:
            fiat_price_without_fee = self.now_sell_fiat_price_without_fee
        return fiat_price_without_fee * (self.trade_unit * unit_amount) >= self.min_trade_fiat_money_limit

    def get_gain_fiat_money(self):
        self.gain_fiat_money = self.now_sell_fiat_price * self.bought_amount - self.used_fiat_money
        return self.now_sell_fiat_price * self.bought_amount - self.used_fiat_money

    def get_total_gain_fiat_money(self):
        return self.total_gained_fiat_money + self.get_gain_fiat_money()

    def __str__(self):
        sell_fiat_price_description = f'{self.now_sell_fiat_price:.4f}'
        if self.bought_average_fiat_price is not None:
            diff_fiat_price = self.now_sell_fiat_price - self.bought_average_fiat_price
            sell_fiat_price_description = f'{diff_fiat_price:.4f} = {self.now_sell_fiat_price:.4f} - {self.bought_average_fiat_price:.4f}'
        return (
            f'GAIN {self.get_total_gain_fiat_money():+.4f} = {self.total_gained_fiat_money:+.4f} {self.get_gain_fiat_money():+.4f} {self.fiat_symbol} in total by {self.get_robot_title()} after\n'
            f'SELL {self.bought_amount:.4f} {self.crypto_symbol} for {sell_fiat_price_description} {self.fiat_symbol}/{self.crypto_symbol}. BUY for {self.now_buy_fiat_price:.4f} {self.fiat_symbol}/{self.crypto_symbol}.\n'
        )

    def get_max_sold_unit_amount(self):
        return self.bought_unit_amount + self.sold_unit_amount

    def estimate_gained_fiat_money(self, sell_unit_amount, sell_fiat_price=None):
        if sell_unit_amount < 0:
            return
        if self.bought_average_fiat_price is None:
            return
        if sell_fiat_price is None:
            sell_fiat_price = self.now_sell_fiat_price
        return (sell_fiat_price - self.bought_average_fiat_price) * (self.trade_unit * sell_unit_amount)

    @classmethod
    def get_status_file(cls):
        return f'{cls.STATUS_FILE}.yaml'

    def get_unique_status_file(self):
        suffix = base64.b64encode(self.robot_name.encode('ascii')).decode('ascii')
        return f'{type(self).STATUS_FILE}-{suffix}.yaml'

    def read(self):
        status_file = self.get_unique_status_file()
        if not os.path.exists(status_file):
            return self
        with open(status_file, 'r') as f:
            self.__dict__.update(
                yaml.load(f, Loader=yaml.FullLoader)
            )
        for i in self.__dict__:
            if isinstance(self.__dict__[i], float):
                self.__dict__[i] = Decimal(repr(self.__dict__[i]))
        return self

    def write(self):
        this = copy.deepcopy(self)
        for i in this.__dict__:
            if isinstance(this.__dict__[i], Decimal):
                this.__dict__[i] = float(this.__dict__[i])
        with open(type(this).TMPFILE, 'w') as f:
            yaml.dump(this.__dict__, f, sort_keys=False)
        os.replace(type(this).TMPFILE, this.get_unique_status_file())

    def clear_bought_status(self):
        self.bought_unit_amount = 0
        self.used_fiat_money = 0
        self.bought_amount = 0
        self.bought_average_fiat_price = None

    def clear_sold_status(self):
        self.sold_unit_amount = 0
        self.got_fiat_money = 0
        self.sold_amount = 0
        self.sold_average_fiat_price = None

    def trim_bought_status(self):
        if self.bought_unit_amount > 0:
            self.bought_amount = self.trade_unit * self.bought_unit_amount
            self.bought_average_fiat_price = self.used_fiat_money / self.bought_amount
        else:
            self.clear_bought_status()

    def trim_sold_status(self):
        if self.sold_unit_amount > 0:
            self.sold_amount = self.trade_unit * self.sold_unit_amount
            self.sold_average_fiat_price = self.got_fiat_money / self.sold_amount
        else:
            self.clear_sold_status()

    def update_bought_status(self, **trade):
        buy_fiat_price = trade.get('fiat_price', self.now_buy_fiat_price)
        buy_unit_amount = trade.get('unit_amount', 0)
        self.bought_unit_amount += buy_unit_amount
        self.used_fiat_money += buy_fiat_price * (self.trade_unit * buy_unit_amount)
        self.trim_bought_status()

    def update_sold_status(self, **trade):
        sell_fiat_price = trade.get('fiat_price', self.now_sell_fiat_price)
        sell_unit_amount = trade.get('unit_amount', 0)
        self.sold_unit_amount += sell_unit_amount
        self.got_fiat_money += sell_fiat_price * (self.trade_unit * sell_unit_amount)
        self.trim_sold_status()

    def trim_trade_status(self):
        self.trim_bought_status()
        self.trim_sold_status()

    def update(self, status=None):
        if isinstance(status, type(self)):
            self.__dict__.update(status.__dict__)
        if isinstance(status, dict):
            self.__dict__.update(status)
        self.trim_fiat_price_status()
        self.trim_trade_status()

In [ ]:
crypto_symbol = 'AR'
fiat_symbol = 'USDT'
crypto_info = {}

Status(
    robot_name=f'{crypto_symbol} Robot',
    crypto_symbol=crypto_symbol,
    fiat_symbol=fiat_symbol,
    max_used_fiat_money_limit=Decimal('1000'),
    **crypto_info
).read().__dict__

{'robot_name': 'AR Robot',
 'crypto_symbol': 'AR',
 'fiat_symbol': 'USDT',
 'crypto_info': {},
 'trade_unit': Decimal('1'),
 'min_trade_fiat_money_limit': Decimal('0'),
 'fee_rate': Decimal('0'),
 'binance_average_fiat_price': 0,
 'max_used_fiat_money_limit': Decimal('1000'),
 'unused_fiat_money': None,
 'now_buy_fiat_price': Decimal('Infinity'),
 'now_sell_fiat_price': 0,
 'now_buy_fiat_price_without_fee': Decimal('Infinity'),
 'now_sell_fiat_price_without_fee': 0,
 'sample_number': 0,
 'trade_count': 0,
 'buy_count': 0,
 'bought_unit_amount': 0,
 'used_fiat_money': 0,
 'bought_amount': 0,
 'bought_average_fiat_price': None,
 'sell_count': 0,
 'sold_unit_amount': 0,
 'got_fiat_money': 0,
 'sold_amount': 0,
 'sold_average_fiat_price': None,
 'total_gained_fiat_money': 0}